<a href="https://colab.research.google.com/github/junhyeongShin/colab/blob/main/%EB%B2%A0%EC%9D%B4%EC%8A%A4%EC%BD%94%EB%93%9C_to_%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 런타임 TPU로 변경하고 사용하기!!!!!
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

VERSION = "20200325"
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION


In [ ]:
pip install torch

In [ ]:
# PyTorch에서는 .to(device) 문법을 통해 텐서 변수들과 모델들을 GPU와 같은 device에 올릴 수 있다.
# TPU에 올리기 위해서는 torch_xla 에서 제공하는 xm.xla_device() 를 통해 PyTorch에 호환되는 device 를 지정할 수 있다.

import torch_xla
import torch_xla.core.xla_model as xm
import torch


#TPU 사용시 꼭 필요한 코드!!!!!!!!!!!!!!
#xm.optimizer_step(optimizer, barrier=True)

import datetime
import os
import time

import numpy as np
import pandas as pd


from matplotlib import pyplot as plt
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive/Colab Notebooks

In [ ]:
pip install Pillow

In [ ]:
from PIL import Image

In [ ]:
data_path = './data/'
train_data_path = os.path.join(data_path, "train")
print(train_data_path)
file_list = os.listdir(train_data_path)
file_list.sort()

In [ ]:
pip install torchvision

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torchvision import transforms, utils
from torchsummary import summary

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm

In [ ]:
path = data_path
train_csv = pd.read_csv(path+'train.csv')
train_csv.head()

In [ ]:
train_path = sorted(glob(path+'train/*.npy'))
train_path = train_path[-30*12:]

train = []
for p in tqdm(train_path):
    # train.append(np.load(p))
    tmp = np.load(p)
    # tmp[:,:,0] = tmp[:,:,0]/250
    train.append(tmp)
    # (360, 448, 304, 5)
train = np.array(train)
train.shape

In [ ]:
print(train[0][250][180][0])

In [ ]:
train_month = []
for i in range(12):
    train_month.append(train[i::12, :,:])
train_month = np.array(train_month)

train_month.shape

In [ ]:
stride = 32
temp = np.zeros([train_month.shape[0],
                 train_month.shape[1],
                 train_month.shape[2],
                 train_month.shape[3]+(stride-train_month.shape[3]%stride),
                 train_month.shape[4]], np.uint8)
temp[:, :,:train_month.shape[2], :train_month.shape[3], :] = train_month
train_month = temp
del temp

In [ ]:
window_size = 5

In [ ]:
x_train = []
y_train = []

for i in range(train_month.shape[1]-window_size):
    x_train.append(train_month[:, i:i+window_size, :,:,:])
    y_train.append(train_month[:, i+window_size:i+window_size+1, :,:,:])
    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
#stride = 32
feature_size = 128
x_train_ = []
y_train_ = []
for y in range(x_train.shape[0]):
    for m in range(12):
        for i in range((x_train.shape[3]-feature_size)//stride+1):
            for j in range((x_train.shape[4]-feature_size)//stride+1):
                x_ = x_train[y, m, :, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size, :1]
                y_ = y_train[y, m, :, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size, :1]
                x_train_.append(x_)
                y_train_.append(y_)

x_train_ = np.array(x_train_)
y_train_ = np.array(y_train_)

x_val_ = []
y_val_ = []

for m in range(12):
    for i in range((x_train.shape[3]-feature_size)//stride+1):
        for j in range((x_train.shape[4]-feature_size)//stride+1):
            x_ = x_train[-1, m, :, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size, :1]
            y_ = y_train[-1, m, :, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size, :1]
            x_val_.append(x_)
            y_val_.append(y_)
            
x_val_ = np.array(x_val_)
y_val_ = np.array(y_val_)

In [ ]:
x_train_.shape, y_train_.shape, x_val_.shape, y_val_.shape

In [ ]:
i = 38
plt.figure(figsize=(15,10))
for y in range(x_train_.shape[1]):
    plt.subplot(1,window_size,y+1)
    plt.imshow(x_train_[i,y,:,:,0])
plt.show()
plt.imshow(y_train_[i,-1,:,:,0])

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, train_x,train_y,batch_size,transform):
        super(TrainDataset, self).__init__()

        self.train_x = train_x
        self.train_y = train_y
        self.batch_size = batch_size
        self.transform = transform

    def __len__(self):
        # 데이터 갯수
        return self.batch_size
    
    def __getitem__(self, idx):

        x = self.train_x[idx*self.batch_size : idx*self.batch_size + self.batch_size]
        y = self.train_y[idx*self.batch_size : idx*self.batch_size + self.batch_size]

        # x = self.train_x
        # y = self.train_y

        x = torch.tensor(x)
        y = torch.tensor(y)

        return x,y



def getTransform():
    return transforms.Compose([transforms.ToTensor()])

transform = getTransform()

#데이터 로더에서 자료 가져오고 배치 사이즈 적용하게끔 하셈
BATCH_SIZE = 32

train_dataset = TrainDataset(x_train_,y_train_,BATCH_SIZE,transform)
valid_dataset = TrainDataset(x_val_,y_val_,BATCH_SIZE,transform)

# Dataloader 클래스는 데이터셋에서 배치 개수만큼 뽑아서 제공해줍니다

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=2, prefetch_factor=2)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=2, prefetch_factor=2)


# show example
# train_dataloader length => 32

# show one of train_ds
for x, y in train_dataset:
    print(x.shape, y.shape)
    break

In [ ]:
m = nn.Conv2d(5,64,3,padding=1)

input = train_dataset.train_x[:32,:5,:,:,0]

input = torch.tensor(input)

input = input.float()

output = m(input)

print(output)

In [ ]:
for x, y in train_dataset:
    print(type(x))
    print(type(y))
    print(x.shape)
    print(y.shape)

    break

In [ ]:
import torch.nn as nn
import torch


class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.
        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class ConvLSTM(nn.Module):

    """
    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.
    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """
        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful
        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param



# The ConvLSTM module derives from nn.Module so it can be used as any other PyTorch module.

# The ConvLSTM class supports an arbitrary number of layers. 
# In this case, it can be specified the hidden dimension (that is, the number of channels) and the kernel size of each layer. 
# In the case more layers are present but a single value is provided, this is replicated for all the layers. 

# For example, in the following snippet each of the three layers has a different hidden dimension but the same kernel size.

# Example usage:

# model = ConvLSTM(input_dim=channels,
#                  hidden_dim=[64, 64, 128],
#                  kernel_size=(3, 3),
#                  num_layers=3,
#                  batch_first=True
#                  bias=True,
#                  return_all_layers=False)        

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

MODEL_PARAMS = {
    "shape" : (5, 1, 128, 128),
    "init_filters": 64,
    "dropout_rate" : 0.5
}

class Net(nn.Module): # nn.Module 모든 신경망 모듈의 기본이 되는 클래스
                      # 각 층과 함수 등 신경망의 구성요소를 이 클래스 안에서 정의한다.
                      # nn.Module은 모든 신경망 모듈의 기본이 되는 클래스로 레이어, 함수등을 정의하는구나!

    def __init__(self):  # 초기화 함수   
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        # (kernel과 filter는 같다) 즉, filter size는 3x3

        input_frames, input_channel, input_height, input_width = params["shape"] # input_frames? input_batch?
        init_filters = params["init_filters"]
        self.dropout_rate = params["dropout_rate"]

        #RNN 모델 적용 필요하다
        self.conv1 = nn.Conv2d(input_channel, 64, 64,padding=1) # 입력 채널 수, 출력 채널 수, 필터의 크기   
        self.batchnorm1 = nn.BatchNorm2d() 
        self.conv2 = nn.Conv2d(64, 64*64, 64) # 입력 채널 수, 출력 채널 수, 필터의 크기    
        self.batchnorm2 = nn.BatchNorm2d()
        self.conv3 = nn.Conv2d(64*64, 64*64*3, 3) # 입력 채널 수, 출력 채널 수, 필터의 크기   
        self.batchnorm3 = nn.BatchNorm2d() 

        self.conv4 = nn.Conv3d(input_channel, 1, kernel_size=3, padding=1)

    def forward(self, x):
        x = conv1(x)
        x = batchnorm1(x)
        x = conv2(x)
        x = batchnorm2(x)        
        x = conv3(x)
        x = batchnorm3(x)
        x = conv4(x)

        return x

# Creating Model

class CustomNet(nn.Module):
    def __init__(self, params):
        super(CustomNet, self).__init__()
        input_frames, input_channel, input_height, input_width = params["shape"] # input_frames? input_batch?
        init_filters = params["init_filters"]
        self.dropout_rate = params["dropout_rate"]
        self.conv1 = nn.Conv3d(input_channel, init_filters, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(init_filters, init_filters*2, kernel_size=3, padding=1)
        self.conv3 = nn.ConvTranspose3d(init_filters*2, 1, kernel_size=3, padding=1)

    def forward(self, x):
        input = x
        x = F.relu(self.conv1(x))
        x = F.max_pool3d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.upsample(x, size=(2, 128, 128))
        print("input: ", input.shape)
        print("output: ", x.shape)
        return x

class test_Net(nn.Module): # nn.Module 모든 신경망 모듈의 기본이 되는 클래스
                      # 각 층과 함수 등 신경망의 구성요소를 이 클래스 안에서 정의한다.
                      # nn.Module은 모든 신경망 모듈의 기본이 되는 클래스로 레이어, 함수등을 정의하는구나!

    def __init__(self):  # 초기화 함수   
        super(test_Net, self).__init__()

        #RNN 모델 적용 필요하다
        self.conv1 = nn.Conv2d(5,64*5, 3,padding=1) # 입력 채널 수, 출력 채널 수, 필터의 크기  
         
        # self.lstm1 = nn.LSTM(64,hidden_size=1,batch_first=True)
        self.batchnorm1 = nn.BatchNorm2d(64*5) 
        # self.conv4 = nn.Conv3d(64, 1, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        # x = self.lstm1(x)
        x = self.batchnorm1(x)
        # x = self.conv4(x)

        return x        


# net = Net()
# print(net)


In [ ]:
class test_Net(nn.Module): # nn.Module 모든 신경망 모듈의 기본이 되는 클래스
                      # 각 층과 함수 등 신경망의 구성요소를 이 클래스 안에서 정의한다.
                      # nn.Module은 모든 신경망 모듈의 기본이 되는 클래스로 레이어, 함수등을 정의하는구나!

    def __init__(self):  # 초기화 함수   
        super(test_Net, self).__init__()

        #RNN 모델 적용 필요하다
        self.conv1 = nn.Conv2d(1,64, 3,padding=1) # 입력 채널 수, 출력 채널 수, 필터의 크기   
        # self.lstm1 = nn.LSTM(64,hidden_size=1,batch_first=True)
        self.batchnorm1 = nn.BatchNorm2d(64) 
        # self.conv4 = nn.Conv3d(64, 1, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        # x = self.lstm1(x)
        x = self.batchnorm1(x)
        # x = self.conv4(x)

        return x 

In [ ]:
model = test_Net()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 
print(model)

In [ ]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(train_dataloader):
    # print(batch_idx)
    # print(len(samples))
    x_train[:32,:], y_train[:32,:] = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

In [ ]:
#TPU 불러오기
device = xm.xla_device()
my_model = test_Net()

# my_model = ConvLSTM(1,hidden_dim=[64, 64, 128],
#                  kernel_size=(3, 3),
#                  num_layers=3,
#                  batch_first=True,
#                  bias=True,
#                  return_all_layers=False) # to(device) 해줘야 에러가 안남
print(my_model)
# 32, 32, 5, 128, 128, 1
my_model.summary()
# summary(my_model, input_size=(32, 5, 128, 128)) # summary 함수를 통해 임의의 사이즈를 넣어 구조와 파라미터를 확인할 수 있습니다

In [ ]:


#모델 생성
net = test_Net()
#TPU 불러오기
device = xm.xla_device()
#모델에 TPU 사용하기
# net.to(device) # to(device) 해줘야 에러가 안남

print(net)
# summary 함수를 통해 임의의 사이즈를 넣어 구조와 파라미터를 확인할 수 있습니다
summary(net, input_size=(1, 128, 128),batch_size=32) 



In [ ]:


#     convLSTM = BatchNormalization()(convLSTM)



#     convLSTM = ConvLSTM2D(filters=64, kernel_size=3, padding='same', return_sequences=True)(convLSTM)

#return_sequences=True의 의미는 
#LSTM의 중간 스텝의 출력을 모두 사용하라는 의미다. 
#그리고 TimeDistributed() 함수의 의미는 각 스텝마다 cost (오류)를 계산해서 하위 스텝으로 
#오류를 전파하여 각 weight를 업데이트하라는 의미다. 


#     convLSTM = BatchNormalization()(convLSTM)

#     convLSTM = ConvLSTM2D(filters=64, kernel_size=3, padding='same', return_sequences=True)(convLSTM)
#     convLSTM = BatchNormalization()(convLSTM)

#     outputs = Conv3D(filters=1, kernel_size=3, activation='relu', padding='same', data_format='channels_last')(convLSTM)

#     model = Model(inputs, outputs)
#     model.compile(loss='mae', optimizer=Adam())


    
# model.summary()

# Model: "model"
# _________________________________________________________________
# Layer (type)                 Output Shape              Param #   
# =================================================================
# input_1 (InputLayer)         [(None, None, 128, 128, 1 0         
# _________________________________________________________________
# conv_lst_m2d (ConvLSTM2D)    (None, None, 128, 128, 64 150016    
# _________________________________________________________________
# batch_normalization (BatchNo (None, None, 128, 128, 64 256       
# _________________________________________________________________
# conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 128, 128, 64 295168    
# _________________________________________________________________
# batch_normalization_1 (Batch (None, None, 128, 128, 64 256       
# _________________________________________________________________
# conv_lst_m2d_2 (ConvLSTM2D)  (None, None, 128, 128, 64 295168    
# _________________________________________________________________
# batch_normalization_2 (Batch (None, None, 128, 128, 64 256       
# _________________________________________________________________
# conv3d (Conv3D)              (None, None, 128, 128, 1) 1729      
# =================================================================
# Total params: 742,849
# Trainable params: 742,465
# Non-trainable params: 384




In [ ]:
# Loss Function && etric Function

# metrics
def mae_score(true, pred):
    true, pred = numpy_to_tensor(true, pred)
    score = np.mean(np.abs(true-pred))
    
    return score

# metrics
def f1_score(true, pred):
    true, pred = numpy_to_tensor(true, pred)

    target = np.where((true>1*0.05)<1*0.5)
    
    true = true[target]
    pred = pred[target]
    true = np.where(true < 1*0.15, 0, 1)
    pred = np.where(pred < 1*0.15, 0, 1)
    
    
    right = np.sum(true * pred == 1)
    precision = right / np.sum(true+1e-8)
    recall = right / np.sum(pred+1e-8)
    score = 2 * precision*recall/(precision+recall+1e-8)
    
    return score
    
# loss function
def mae_over_f1(true, pred):
    mae = mae_score(true, pred)
    f1 = f1_score(true, pred)
    score = mae/(f1+1e-8)
    
    return score

def numpy_to_tensor(true, pred):
    return true.cpu().detach().numpy(), pred.cpu().detach().numpy()

In [ ]:
# Optimizer

# adam optimizer
opt_adam = optim.Adam(my_model.parameters(), lr=3e-4)

def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group["lr"]

# check our learning rate
current_lr = get_lr(opt_adam)
print(f"current_lr = {current_lr}")



# learning rate scheduler
lr_scheduler = ReduceLROnPlateau(opt_adam, mode="min", factor=0.5, patience=20, verbose=1)

# example 
for i in range(100):
    lr_scheduler.step(1)

In [ ]:
# Training 

def metrics_batch(pred, true, metrics):
    # if needed add param "metrics" to custom
    """
    output will be pred
    target will be corrects
    """
    if metrics:
        return list(map(lambda x: x(true, pred), metrics))
    mae_score = mae_score(true, pred)
    f1_score = f1_score(true, pred)
    return (mae_score, f1_score)

def loss_batch(loss_func, pred, true, opt=None):
    """
    loss_func => mae_over_f1
    """
    loss = loss_func(true, pred)
    with torch.no_grad():
        metrics = metrics_batch(pred, true, [mae_score, f1_score])
    if opt is not None:
        opt.zero_grad()
        # loss.backward()
        opt.step()  # 학습이 이뤄지는 곳
    return loss, metrics

def loss_epoch(model, loss_func, dataset_dataloader, sanity_check=False, opt=None):
    running_loss = 0.0
    running_metric = [0.0, 0.0]
    len_data = len(dataset_dataloader.dataset)

    for x, y in dataset_dataloader:
        x = x.to(device)
        y = y.to(device)
        # 모델 결과
        pred = model(x)
        # 손실함수 구하기
        loss, metrics = loss_batch(loss_func, pred, y, opt)
        # 손실함수 
        running_loss += loss
        if metrics is not None:
            for idx, metric_value in enumerate(metrics):
                running_metric[idx] += metric_value
        
        # 문제 있으면 break, 여기서는 True 일때 바로 break
        if sanity_check is True:
            break
    
    loss = running_loss / float(len_data)
    metrics = list(map(lambda x: x/float(len_data), metrics))
    print(loss, metrics)
    return loss, metrics

In [ ]:
loss_func = mae_over_f1
opt_adam = optim.Adam(my_model.parameters(), lr=3e-4)
lr_scheduler = ReduceLROnPlateau(opt_adam, mode="min", factor=0.5, patience=20, verbose=1)

TRAIN_PARAMS = {
    "num_epochs" : 10,
    "loss_func" : loss_func,
    "optimizer" : opt_adam,
    "train_dataloader" : train_dataloader,
    "valid_dataloader" : valid_dataloader,
    "sanity_check" : True,
    "lr_scheduler" : lr_scheduler,
    "save_path" : "./weights.pt"
}

In [ ]:
import copy

def train(model, params):
    num_epochs = params['num_epochs']
    loss_func = params['loss_func']
    opt = params["optimizer"]
    train_dataloader = params['train_dataloader']
    valid_dataloader = params['valid_dataloader']
    sanity_check = params['sanity_check']
    lr_scheduler = params['lr_scheduler']
    save_path = params['save_path']

    # keep history of the loss and metric
    loss_hist = {
        "train" : [],
        "valid" : []
    }

    metrics_hist = {
        "train" : [],
        "valid" : []
    }

    # copy best weights
    best_model_weights = copy.deepcopy(model.state_dict())
    # init best loss
    best_loss = float("inf")

    for epoch in range(num_epochs):
        current_lr = get_lr(opt)
        print(f'Epoch:{epoch}/{num_epochs-1}, current lr:{current_lr}')
        model.train()
        train_loss, train_metrics = loss_epoch(model, loss_func, train_dataloader, sanity_check, opt)

        # save history
        loss_hist["train"].append(train_loss)
        metrics_hist["train"].append(train_metrics)

        # model.eval()
        # with torch.no_grad():
    

    return model, loss_hist, metrics_hist

In [ ]:
my_model, loss_hist, metrics_hist = train(my_model, TRAIN_PARAMS)

print(loss_hist)